In [113]:
# Load Data
%%bash
pip install pyspark
# Download the data files from github
# If the data file does not exist in the colab environment
if [[ ! -f ./players_20.csv ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/Kun97/FIFA20_DataAnalysis/main/players_20.csv
fi

In [171]:
# The first code cell of your notebook shall include all needed imports to run your project code.  Note that
# there can be markdown cells above this cell.
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, lit, split, isnan, when, count, isnull
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexerModel, StringIndexer, OneHotEncoder

In [115]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [142]:
original_data = spark.read.csv("players_20.csv", inferSchema=True, header=True)

In [143]:
# drop useless columns
drop1_columns = ['sofifa_id', 'player_url', 'long_name', 'player_tags', 'player_traits', 'real_face', 'nation_jersey_number', 'team_jersey_number', 'loaned_from', 'joined', 'contract_valid_until']
fixed1_Data = original_data.drop(*drop1_columns)

In [144]:
# drop position ability(duplicated and will affect the final result)
drop2_columns = fixed1_Data.columns[-26:]
fixed2_Data = fixed1_Data.drop(*drop2_columns)

In [145]:
# drop duplicated columns
drop3_columns = ['wage_eur', 'team_position', 'release_clause_eur', 'nation_position', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
fixed3_Data = fixed2_Data.drop(*drop3_columns)

In [146]:
# split player_positions and choose the first position as primary position
split_col = split(fixed3_Data['player_positions'], ',')
position_data = fixed3_Data.withColumn('Position', split_col.getItem(0))
position_data = position_data.drop('player_positions')

In [156]:
# split work_Rate 
split_col = split(position_data['work_rate'], '/')
position_data = position_data.withColumn('Attack_workrate', split_col.getItem(0))
position_data = position_data.withColumn('Defend_workrate', split_col.getItem(1))
position_data = position_data.drop('work_rate')

In [159]:
# narrow down positions: defender
defender = ['LCB', 'RCB', 'LB', 'RB', 'CB', 'RWB', 'LWB']
defender_data = position_data.filter(col('Position').isin(defender)).withColumn('Positon_General', lit('Defender'))
defender_data.count()

5938

In [160]:
# midfileder
midfileder = ['LCM', 'LM', 'RDM', 'CAM', 'RAM', 'RCM', 'CM', 'CDM', 'RM', 'LAM', 'LDM']
midfileder_data = position_data.filter(col('Position').isin(midfileder)).withColumn('Positon_General', lit('Midfileder'))
midfileder_data.count()

6862

In [161]:
# attacker
attacker = ['ST', 'CF', 'LW', 'RW']
attacker_data = position_data.filter(col('Position').isin(attacker)).withColumn('Positon_General', lit('Attacker'))
attacker_data.count()

3442

In [162]:
# goal keeper
gk_data = position_data.filter(col('Position').isin(['GK'])).withColumn('Positon_General', lit('GK'))
gk_data.count()

2036

In [163]:
general_data = defender_data.union(midfileder_data).union(attacker_data).union(gk_data)

In [152]:
# missing values: 
general_data.select([count(when(isnull(c), c)).alias(c) for c in general_data.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------------+------------------------+---------+-----------+---------+---------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+------------------+--------------------+-------------------+-----------------------+--------------------+--------+---------------+
|short_name|age|dob|height_cm|weight_kg|nationality|club|overall|potential|value_eur|preferred_foot|international_reputation|weak_foot|skill_mo

In [153]:
# duplicated values: general data
if general_data.count() == general_data.drop_duplicates().count():
  print('No duplicated records.')

No duplicated records.


In [213]:
# numerlize categorical data
# Rate: low, medium, high: 0, 1, 2
# foot: left, right: 1, 0
data_prepipe = Pipeline(stages=[
                                StringIndexerModel.from_labels(['Low', 'Medium', 'High'], inputCol='Attack_workrate', outputCol='Attack_workrate_num'),
                                StringIndexerModel.from_labels(['Low', 'Medium', 'High'], inputCol='Defend_workrate', outputCol='Defend_workrate_num'),
                                StringIndexer(inputCols=["preferred_foot", "body_type", "Positon_General"], outputCols=["preferred_foot_num", "body_type_num", "Positon_General_num"]),
                                OneHotEncoder(inputCols=['preferred_foot_num', 'body_type_num'], outputCols=['foot', 'body'])
])

In [214]:
general_data_xformed = data_prepipe.fit(general_data).transform(general_data)

In [215]:
general_data_xformed.show()

+---------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------------+------------------------+---------+-----------+---------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+------------------+--------------------+-------------------+-----------------------+--------------------+--------+---------------+---------------+---------------+-------------------+-------------------+------------------+-------------+-------------------+--